# Raspberrypi4 Install

We can install the operating systems:
- Raspberry Pi OS
- Ubuntu 18 server

## Install Raspberry Pi OS
Follow the steps in order to properly install the Raspberrypi:

- Install Raspberry Pi OS using Raspberry Pi Imager (download for windows): https://www.raspberrypi.org/software/
- Run the application and save the image:
    - Raspbian: (Raspberry pi OS (32-bit)) to the SD card
- Insert the SD in a RBPi board and connect it to screen
- The system ask you to choose a wifi network and change the timezone, language and password
- Reboot and select Raspberry Pi Configuration to activate the SSH and Camera
- reboot and open a terminal to obtain the IP address using "ifconfig"

You are now ready to connect from your Ubuntu Virtual machine using VS Code or a simple terminal

<img src="./Images/5_rbpi4_imager.png">

Alternativelly you can connect to the RBPi using SSH in a wifi network without the previous configurations using a screen connection if you follow the instructions:

- add ssh file (without extension) in the SD in the G:boot directory
- add wpa_supplicant.conf file containing different wifi information

## Ubuntu 18 server on RaspberryPi4

Follow the steps:
- Download the image Ubuntu 18 server for Raspberrypi3 (64-bit ARM): https://cdimage.ubuntu.com/releases/18.04/release/
- Use Balena Etcher to copy it to a SD card 32GB
- connect the RaspberryPi4 with internet cable and insert the SD card
- Connect to a Screen and after few minutes press enter
- type "ubuntu" for login and "ubuntu" for password
- modify the password
- type "sudo apt update"
- type "sudo apt upgrade"
- type "sudo apt-get install ubuntu-desktop" (or xubuntu-desktop)
- type "sudo reboot"


### Win IP Scanner
Install IP scanner for windows: https://www.advanced-ip-scanner.com/es/

Connect the RBPi to the router and find the IP number

### Connection to RBPi from Ubuntu PC
The first connection to the RBPi has to be using a Terminal in Virtual Box:

- type "ssh pi@192.168.18.46" (the default user:pi and pass:raspberry)
- When you connect the same RBPi to another network, you have a warning because you have to regenerate the KeyGen of your computer. You have to type (using the proper IP): ssh-keygen -R 192.168.18.46

### Remote Desktop 
This Desktop is available for Windows and MAC users. To configure this remote access:
#### In Raspberry Pi OS: VNC

- To begin, open up the raspi-config tool by running the following command: sudo raspi-config
- Within the configuration tool, use the arrow keys to scroll down to "2. Display Options" and "D1. Resolution"
- Select the desired resolution. Usually DMT Mode 82 1920x1080 60Hz 16:9
- Press OK
- Within the configuration tool, use the arrow keys to scroll down to "3. Interface Options" and "P1. Camera" and "P3. VNC"
- Select VNC enable
- Press "finish" and "OK" and reboot

In any OS you will have to:

Now in your PC install VNC Viewer: https://www.realvnc.com/es/connect/download/viewer/raspberrypi/

You will be able to enter in the raspberrypi4 desktop

#### In Ubuntu 18 server: nomachine

- Download nomachine in RaspberryPi and PC: 
    - In raspberryPi (ARMv8 DEB): https://www.nomachine.com/download/linux&id=29&s=Raspberry
    - In PC: https://www.nomachine.com/
    

### Assign a static IP address

You need to modify the "/etc/dhcpcd.conf" file with the last following lines:

In [ ]:
interface wlan0
static ip_address=192.168.18.200/24
static routers=192.168.18.1
static domain_name_servers=192.168.18.1

Carefull!:
- select the interface wifi: in my case wlan0
- select the router domain in my case is: 192.168.18.15/24 (here the desired static IP is 15) 

## Install ROS Melodic
The installation will be in function of the OS installed on raspberrypi:
- Raspbian
- Ubuntu18

### Raspbian
Follow the instructions on the document: Raspbian_ROS_Melodic.txt

Refer to the official webpage: http://wiki.ros.org/ROSberryPi/Installing%20ROS%20Melodic%20on%20the%20Raspberry%20Pi

### Ubuntu
Follow the instructions on: http://wiki.ros.org/melodic/Installation/Ubuntu

## Create our workspace

We will create the workspace where we will install all needed packages for our Hardware project

In [ ]:
mkdir -p ~/rubot_ws/src
cd ~/rubot_ws/
catkin_make
echo "source ~/rubot_ws/devel/setup.bash" >> ~/.bashrc

### Install Sensor packages
#### 1. Camera RaspberryPi

In [ ]:
cd ~/rubot_ws/src/
mkdir camera
cd ~/rubot_ws/src/camera/
git clone https://github.com/UbiquityRobotics/raspicam_node.git
git clone --single-branch --branch=indigo-devel https://github.com/ros-perception/image_transport_plugins.git
git clone --single-branch --branch=melodic https://github.com/ros-perception/vision_opencv.git
git clone --single-branch --branch=hydro-devel https://github.com/ros-perception/image_common.git

#### 2. LIDAR

We will install the 2 LIDAR models we have: YD-LIDAR and RP-LIDAR

In [ ]:
cd ~/rubot_ws/src/
mkdir lidar

##### YD-LIDAR

In [ ]:
cd ~/rubot_ws/src/lidar/
git clone https://github.com/EAIBOT/ydlidar.git

Plug the LIDAR USB and we want pi and dialout as owner/owner group

In [ ]:
ls -la /dev/ttyUSB*
sudo chown pi:dialout /dev/ttyUSB0
roscd ydlidar/startup
sudo chmod a+rwx ./*
sudo sh initenv.sh

##### RP-LIDAR

In [ ]:
cd ~/rubot_ws/src/lidar/
git clone https://github.com/Slamtec/rplidar_ros

#### 4. Mecanum robot

This robot will be controlled by an Arduino Mega board.

The arduino program will start a serial_node with all the topics

In [ ]:
git clone https://github.com/ros-gopigo/gopigo3_node
git clone --single-branch --branch=kinetic-devel https://github.com/ros-teleop/teleop_tools.git

## Setup rUBot with sensors
We will execute in different terminals:

- the rubot_mecanum node for driving control
- the ydlidar node
- the raspicam node

In [ ]:
roslaunch mecanum_control rubot_control1.launch
roslaunch ydlidar lidar.launch
roslaunch raspicam_node camerav2_1280x960_10fps.launch enable_raw:=true camera_frame_id:="laser_frame"

<img src="./Images/5_rubot_nodes.png">